# Import Libraries

In [1]:
import pandas as pd
from rapidfuzz import process, fuzz
import re
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
#from joypy import joyplot 

# Import Files

Importing one excel file called **Historical_FutureSales** with two tabs, the first tab is for shipments made so far and the second tab lists our open orders.  The tab names must be:

1. Shipments
2. OpenOrders

In [2]:
file_path = "Historical_FutureSales.xlsx"
df_historic = pd.read_excel(file_path, sheet_name="Shipments")
df_openOrders= pd.read_excel(file_path, sheet_name="OpenOrders")

## Cleaning the Historical Sales Tab

The unnecessary columns have been eliminated and the remaining columns will be:

1. Date             datetime64[ns]
2. Company                  object
3. Aspacs                   object
4. Base                     object
5. Piece Price             float64
6. Total Sales             float64
7. Currency                 object
8. Exchange Rate           float64

In [4]:
df_historic_clean = df_historic.drop(['NoFacture', 'Catégorie',
       'Regroupement','Serie', 'Capacité statistique'], axis=1).rename(columns={'DateDeTransfertEnCA': 'Date', 
                                                                        'Nom entreprise': 'Company', 
                                                                        'Monnaie': 'Currency',
                                                                        'MonnaieCours': 'Exchange Rate', 
                                                                        'Qté': 'Quantity', 
                                                                        'PrixFact': 'Piece Price', 
                                                                        'MontPosCHF': 'Total Sales', 
                                                                        'Pays': 'Country' 
                                                                        })
df_historic_clean = df_historic_clean[['Date', 'Company', 'Aspacs', 'Base','Piece Price', 'Total Sales', 'Currency', 'Exchange Rate']]
# df_historic_clean['Date'] = df_historic_clean['Date'].dt.date
df_historic_clean = df_historic_clean.round({'Total Sales': 2, 'Exchange Rate': 2})
# df_historic_clean['Total Sales'] = df_historic_clean['Total Sales'].apply(lambda x: "{:,.2f}".format(x))

## Cleaning the Open Orders Tab

The unnecessary columns have been eliminated and the remaining columns will be:

1. Date             datetime64[ns]
2. Company                  object
3. Aspacs                   object
4. Piece Price             float64
5. Total Sales             float64
6. Currency                 object
7. Exchange Rate           float64

In [5]:
df_openOrders_clean = df_openOrders.drop(['Année', 'Mois', 'Délai', 'DateFinContrat', 'Catégorie', 'NoCommande',
       'NoPos', 'CondSurAppel', 'CatCom'], axis=1).rename(columns={'DélaiStat': 'Date', 
                                                                   'NoClient':"ClientID",
                                                                   'Qté': 'Quantity', 
                                                                   'Prix': 'Piece Price', 
                                                                   'MontChfAct': 'Total Sales', 
                                                                   'Monnaie': 'Currency', 
                                                                   'cours': 'Exchange Rate',
                                                                   'Nom entreprise': 'Company'
                                                                      })

#display(df_openOrders_clean)

df_openOrders_clean = df_openOrders_clean[['Date','ClientID', 'Company', 'Aspacs', 'Piece Price', 'Total Sales', 'Currency', 'Exchange Rate']]
df_openOrders_clean['Date'] = df_openOrders_clean['Date'].dt.date
#df_openOrders_clean = df_openOrders_clean.round({'Total Sales': 2, 'Exchange Rate': 2})
#df_openOrders_clean['Total Sales'] = df_openOrders_clean['Total Sales'].apply(lambda x: "{:,.2f}".format(x))

In [6]:
df_openOrders_clean.head()

,Date,ClientID,Company,Aspacs,Piece Price,Total Sales,Currency,Exchange Rate
0,2024-09-28,1663,Tam han,JO-608-W5 WW,0.16,416.000000,CHF,1.0000
1,2024-09-30,1663,sam bam,X814-W2 WW,5.25,32550.000000,CHF,1.0000
2,2024-09-29,1663,KFC,608-W22 WW,1.83,51240.000000,CHF,1.0000
3,2024-09-29,32011,KFC Other,1203SD_SKF-W1 WW,4.96,13193.600000,CHF,1.0000
4,2024-10-01,1679,Some Tam,16100-2RSRY RW D8SS5,3.89,2036.169152,EUR,0.9248


### Adding extra random data features (Data Augumentation)

- Here we will add random data features to check the chart results

In [160]:
min_data = pd.to_datetime("2018-09-28")
max_data = pd.to_datetime("2024-10-02")

In [161]:
import random
from datetime import timedelta
import numpy as np

def random_date(start_date, end_date):
    delta = end_date - start_date
    random_days = random.randint(1, delta.days)
    
    date = start_date + timedelta(days=random_days)
    
    return date.date()


def augment_data(df, n_rows):
    augmented_data = []
    
    for _ in range(n_rows):
        
        #The choices in each itration will be random and unique fromm the same feature
        client_id = random.choice(df['ClientID'].unique())
        company = random.choice(df['Company'].unique())
        aspacs = random.choice(df['Aspacs'].unique())
        piece_price = round(np.random.uniform(0.1, 10), 2)
        total_sales = round(piece_price * random.randint(50, 5000), 2)
        date = random_date(min_data, max_data)
        currency = random.choice(df['Currency'].unique())

        if currency == 'CHF':
            exchange_rate = 1.0000
        else:
            exchange_rate = round(np.random.uniform(0.9, 1.2), 4)  
        
        augmented_data.append([date, client_id, company, aspacs, piece_price, total_sales, currency, exchange_rate])
    
    augmented_df = pd.DataFrame(augmented_data, columns=['Date', 'ClientID', 'Company', 'Aspacs', 'Piece Price', 'Total Sales', 'Currency', 'Exchange Rate'])
    
    
    return pd.concat([df, augmented_df], ignore_index=True)


augmented_df = augment_data(df_openOrders_clean, 100)


augmented_df

,Date,ClientID,Company,Aspacs,Piece Price,Total Sales,Currency,Exchange Rate
0,2024-09-28,1663,Tam han,JO-608-W5 WW,0.16,416.000000,CHF,1.0000
1,2024-09-30,1663,sam bam,X814-W2 WW,5.25,32550.000000,CHF,1.0000
2,2024-09-29,1663,KFC,608-W22 WW,1.83,51240.000000,CHF,1.0000
3,2024-09-29,32011,KFC Other,1203SD_SKF-W1 WW,4.96,13193.600000,CHF,1.0000
4,2024-10-01,1679,Some Tam,16100-2RSRY RW D8SS5,3.89,2036.169152,EUR,0.9248
...,...,...,...,...,...,...,...,...
100,2019-10-29,32011,KFC Other,608-W22 WW,1.12,3543.680000,CHF,1.0000
101,2023-12-25,32011,KFC Other,JO-608-W5 WW,4.27,8232.560000,EUR,1.0606
102,2020-05-19,1679,KFC,X814-W2 WW,5.32,14784.280000,CHF,1.0000
103,2024-03-15,1663,sam bam,16100-2RSRY RW D8SS5,9.50,31768.000000,EUR,1.1848


In [162]:
from collections import Counter
Counter(augmented_df["Company"])

Counter({'Tam han': 25,
         'sam bam': 21,
         'KFC': 23,
         'KFC Other': 20,
         'Some Tam': 16})

## List of Possible Duplicates

This block combines company names that are similar but do not have the same example spell listed for the company.  It uses the **rapidfuzzy** library to evaluate a similarity score and create a list of duplicates. 

The following words will be ignored when making the evaluaion since the can scew the similarity score:

1. "Deutschland"
2. "International"
3. "Espagne"

### Possible Duplicates of Historical Sales

In [163]:
# Words to ignore during scoring
ignore_words = ["Deutschland", "International", "Espagne"]

# Function to clean company names
def clean_name(name, words_to_ignore):
    if pd.isna(name):
        return ''  # Handle None or NaN values
    # Create a regex pattern to remove words to ignore
    pattern = r'\b(?:' + '|'.join(words_to_ignore) + r')\b'
    # Remove the ignored words and any extra spaces left behind
    return re.sub(pattern, '', name, flags=re.IGNORECASE).strip()

# Function to find similar company names without specific words in scoring
def find_similar_names(df, column_name, threshold=80):
    company_names = df[column_name].tolist()  # Get all the company names
    cleaned_names = [clean_name(name, ignore_words) for name in company_names]  # Preprocess to remove ignored words
    
    similar_groups = []  # List to store groups of similar names
    processed = set()    # Set to keep track of processed names

    # Compare each company name to all others
    for i, name in enumerate(company_names):
        if name in processed:
            continue

        # Find matches for the cleaned name
        matches = process.extract(cleaned_names[i], cleaned_names, scorer=fuzz.token_sort_ratio, limit=None)

        # Extract only the original names corresponding to the matches, excluding exact matches
        matched_names = [company_names[j] for match, score, j in matches if score >= threshold and j != i and cleaned_names[i].lower() != cleaned_names[j].lower()]

        if matched_names:
            similar_groups.append([name] + matched_names)

        # Add the original names that were matched to the processed set
        processed.update(matched_names)
        processed.add(name)

    return similar_groups

# Use the function on the 'Company' column with a similarity threshold of 70%
similar_name_groups = find_similar_names(df_historic_clean, 'Company', threshold=70)

# Display the results
for group in similar_name_groups:
    print(group)

['Company A', 'Company B', 'Company C', 'Company B', 'Company B', 'Company D', 'Company D']
['Copmany E', 'Company D', 'Company D']


### Possible Duplicates of Future Open Orders

In [164]:
# Use the function on the 'Company' column with a similarity threshold of 70%
similar_name_groups = find_similar_names(augmented_df, 'Company', threshold=50)

print(similar_name_groups)

# Display the results
for group in similar_name_groups:
    print(group)

[['Tam han', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam'], ['KFC', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other', 'KFC Other'], ['Some Tam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam']]
['Tam han', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam', 'sam bam'

## Rename Similar Companies

This block take the lists of company names created in the previous block and renames all the companies classified as simliar and renames them to the first name in the list.  For example *LAMAICA PEARINGS CO CAMARILLO (CA)* and *LAMAICA PEARINGS CO INC.* will be classified as the same and called *LAMAICA PEARINGS CO CAMARILLO (CA)*.

### Renaming Historical Sales File

In [165]:
# Step 1: Create a mapping dictionary
name_mapping = {}

# Iterate over each group of similar names
for group in similar_name_groups:
    # The first name in the group will be the standard name
    standard_name = group[0]
    # Map all names in the group to the standard name
    for variant in group:
        name_mapping[variant] = standard_name

# Step 2: Replace company names in the original DataFrame
#Assuming `df_historic_clean` is your DataFrame containing the 'Company' column


#df_historic_clean['Company'] = df_historic_clean['Company'].replace(name_mapping)

# Display the updated DataFrame
# print(df_historic_clean)

In [166]:
name_mapping

{'Tam han': 'Tam han',
 'sam bam': 'Some Tam',
 'KFC': 'KFC',
 'KFC Other': 'KFC',
 'Some Tam': 'Some Tam'}

### Renaming Open Order File

In [167]:
# Assuming `df_openOrders_clean` is your DataFrame containing the 'Company' column
augmented_df['Company'] = augmented_df['Company'].replace(name_mapping)

In [168]:
df_historic_clean.head(5)

,Date,Company,Aspacs,Base,Piece Price,Total Sales,Currency,Exchange Rate
0,2024-09-26 08:22:27,Company A,JO-61902-W3 WW,JO-ForSales,0.24,2219.52,EUR,0.92
1,2024-09-26 07:50:37,Company A,OT8MSD01-W2 WW,3B27,6.05,3630.00,CHF,1.00
2,2024-09-23 11:39:23,Company B,SPLFR50/5-2RSRT9H WW TN5S6,3A51,7.04,14410.88,CHF,1.00
3,2024-09-23 11:39:23,Company C,SPLFR50/5-2RSRT9H WW TN5S6,3A51,7.04,19296.64,CHF,1.00
4,2024-09-23 11:39:23,Company D,3002-W2 WW,3002,7.48,15842.64,CHF,1.00


# 1) Working on openorder dataset

In [169]:
augmented_df.head(5)

,Date,ClientID,Company,Aspacs,Piece Price,Total Sales,Currency,Exchange Rate
0,2024-09-28,1663,Tam han,JO-608-W5 WW,0.16,416.000000,CHF,1.0000
1,2024-09-30,1663,Some Tam,X814-W2 WW,5.25,32550.000000,CHF,1.0000
2,2024-09-29,1663,KFC,608-W22 WW,1.83,51240.000000,CHF,1.0000
3,2024-09-29,32011,KFC,1203SD_SKF-W1 WW,4.96,13193.600000,CHF,1.0000
4,2024-10-01,1679,Some Tam,16100-2RSRY RW D8SS5,3.89,2036.169152,EUR,0.9248


In [170]:
augmented_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           105 non-null    object 
 1   ClientID       105 non-null    int64  
 2   Company        105 non-null    object 
 3   Aspacs         105 non-null    object 
 4   Piece Price    105 non-null    float64
 5   Total Sales    105 non-null    float64
 6   Currency       105 non-null    object 
 7   Exchange Rate  105 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 6.7+ KB


### We need to check some Nans and data types

- Date feature needs to be date and time format
- for our float features like total sales we need to convert it into float type in order to calculate the total prices for company and client

In [171]:
augmented_df['Date']  = pd.to_datetime(augmented_df['Date'])

In [172]:
#augmented_df['Total Sales'] = augmented_df['Total Sales'].str.replace(',', '').astype(float)
augmented_df

,Date,ClientID,Company,Aspacs,Piece Price,Total Sales,Currency,Exchange Rate
0,2024-09-28,1663,Tam han,JO-608-W5 WW,0.16,416.000000,CHF,1.0000
1,2024-09-30,1663,Some Tam,X814-W2 WW,5.25,32550.000000,CHF,1.0000
2,2024-09-29,1663,KFC,608-W22 WW,1.83,51240.000000,CHF,1.0000
3,2024-09-29,32011,KFC,1203SD_SKF-W1 WW,4.96,13193.600000,CHF,1.0000
4,2024-10-01,1679,Some Tam,16100-2RSRY RW D8SS5,3.89,2036.169152,EUR,0.9248
...,...,...,...,...,...,...,...,...
100,2019-10-29,32011,KFC,608-W22 WW,1.12,3543.680000,CHF,1.0000
101,2023-12-25,32011,KFC,JO-608-W5 WW,4.27,8232.560000,EUR,1.0606
102,2020-05-19,1679,KFC,X814-W2 WW,5.32,14784.280000,CHF,1.0000
103,2024-03-15,1663,Some Tam,16100-2RSRY RW D8SS5,9.50,31768.000000,EUR,1.1848


### Checking top 10 grossing customers/companies

In [173]:
grouped_data_company = pd.DataFrame(augmented_df.groupby(["Company", "Date"])["Total Sales"].sum())

In [174]:
grouped_data_client = pd.DataFrame(augmented_df.groupby(["ClientID", "Date"])["Total Sales"].sum())

In [175]:
grouped_data_company = grouped_data_company.reset_index()
grouped_data_company

,Company,Date,Total Sales
0,KFC,2018-10-13,11913.80
1,KFC,2018-12-24,3910.45
2,KFC,2019-01-05,11150.75
3,KFC,2019-06-27,28228.44
4,KFC,2019-07-22,4750.59
...,...,...,...
99,Tam han,2023-06-03,2891.14
100,Tam han,2023-10-19,2105.16
101,Tam han,2024-02-27,4074.72
102,Tam han,2024-09-01,471.00


In [176]:
grouped_data_client = grouped_data_client.reset_index()
grouped_data_client

,ClientID,Date,Total Sales
0,1663,2018-10-13,11913.80
1,1663,2018-11-13,368.88
2,1663,2018-11-20,1654.24
3,1663,2018-12-24,3910.45
4,1663,2019-01-05,11150.75
...,...,...,...
100,32011,2024-03-03,17393.04
101,32011,2024-05-11,5432.96
102,32011,2024-07-08,6804.00
103,32011,2024-09-18,9672.00


### Open orders dataset | Visualization

In [177]:
import warnings 
warnings.filterwarnings("ignore")

### Overall company sales over the time period


- we can filter out the companies here as well
- and also we can filter out the year range from the features


In [178]:
import plotly.express as px
augmented_df=augmented_df.sort_values(by="Date")

fig = px.line(
    augmented_df,
    x='Date',              
    y='Total Sales',       
    color='Company',       
    markers=True,          
    title='Total Sales by Company Over Time',
    labels={'Total Sales': 'Total Sales (CHF)', 'Date': 'Date'}
)


fig.update_layout(
    xaxis_tickformat='%Y-%m-%d',   
    xaxis_title='Date',
    yaxis_title='Total Sales',
    legend_title='Company',
    title_x=0.5,                   
    plot_bgcolor='rgba(0,0,0,0)',  
    hovermode='x unified',         
)


fig.show()

### Customized plot for specific company over the time sales details

In [179]:
import plotly.express as px

def customized_company_details(grouped_data_company,comp, date_min, date_max):
    
    filted_data =grouped_data_company[grouped_data_company["Company"] == comp]
    filtered_data = filted_data[(filted_data["Date"]> date_min) & (filted_data["Date"]< date_max)]
    #display(filtered_data)

    fig = px.bar(
        filtered_data,
        x='Date',          
        y='Total Sales',   
        color='Company',   
        title="Total Sales by Company and Date",
        labels={'Total Sales': 'Total Sales', 'Date': 'Date'},
        barmode='group',   
        text='Total Sales' 
    )


    fig.update_traces(
        texttemplate='%{text:.2f}',   
        textposition='outside',       
    )
    
    fig.update_xaxes(
    tickangle=-45  
    )



    fig.update_layout(
        xaxis_tickformat='%Y-%m-%d',   
        xaxis_title='Date',
        yaxis_title='Total Sales (CHF)',
        legend_title='Company',
        title_x=0.5,                   
        plot_bgcolor='rgba(0,0,0,0)',  
        bargap=0.2                     
    )


    fig.show()

### Input your customized company details

In [180]:
## You can chnage these input feature from your filters as well
comp = "KFC"
date_min = pd.to_datetime("2024-04-01")
date_max = pd.to_datetime("2024-09-28")


customized_company_details(grouped_data_company,comp, date_min, date_max)

### Finding top 5 clients in the dataset

In [181]:
grouped_data_client

,ClientID,Date,Total Sales
0,1663,2018-10-13,11913.80
1,1663,2018-11-13,368.88
2,1663,2018-11-20,1654.24
3,1663,2018-12-24,3910.45
4,1663,2019-01-05,11150.75
...,...,...,...
100,32011,2024-03-03,17393.04
101,32011,2024-05-11,5432.96
102,32011,2024-07-08,6804.00
103,32011,2024-09-18,9672.00


In [182]:
set(grouped_data_client.ClientID)

{1663, 1679, 32011}

In [183]:
grouped_data_client_group = grouped_data_client.groupby("ClientID")["Total Sales"].sum()
grouped_data_client_group = grouped_data_client_group.reset_index()

In [184]:
top_5_clients = grouped_data_client_group.sort_values(by='Total Sales', ascending=False).head(5)

In [185]:
top_5_clients["ClientID"] =top_5_clients["ClientID"].astype(str) 

### The following visualization shows the total sum for each client

- It will sort top 5 clients in the dataset with heighest of the total sums 

In [186]:
fig = px.bar(
    top_5_clients,
    x='ClientID',          
    y='Total Sales',   
    #color='Company',   
    title="Total Sales by Clients",
    labels={'Total Sales': 'Total Sales', 'Date': 'ClientsID'},
    #barmode='group',   
    text='Total Sales' 
)


fig.update_traces(
    texttemplate='%{text:.2f}',   
    textposition='outside',       
)


fig.update_layout(
    xaxis_tickformat='%Y-%m-%d',   
    xaxis_title='Client IDs',
    yaxis_title='Total Sales',
    #legend_title='Clients',
    title_x=0.5,                   
    plot_bgcolor='rgba(0,0,0,0)',  
    bargap=0.2                     
)


fig.show()

### Search for specific client and check its total sales over the time

- the following line charsts shows the daily total sums over the given time interval

In [187]:
def get_specific_client_Details(grouped_data_client, client_id, start, end):

    grouped_data_client=grouped_data_client.sort_values(by="Date")
    grouped_data_client["ClientID"] =grouped_data_client["ClientID"].astype(str) 
    
    
    filted_data = grouped_data_client[grouped_data_client["ClientID"].isin(client_id)]
    filtered_data = filted_data[(filted_data["Date"]> start) & (filted_data["Date"]< end)]

    #display(filtered_data)
    fig = px.line(
        filtered_data,
        x='Date',              
        y='Total Sales',       
        color='ClientID',       
        markers=True,          
        title='Total Sales by Client Over Time',
        labels={'Total Sales': 'Total Sales', 'Date': 'Date'}
    )
    
    fig.update_xaxes(
    tickangle=-45  
    )


    fig.update_layout(
        xaxis_tickformat='%Y-%m-%d',   
        xaxis_title='Date',
        yaxis_title='Total Sales',
        legend_title='Client ID',
        title_x=0.5,                   
        plot_bgcolor='rgba(0,0,0,0)',  
        hovermode='x unified',  
        
    )


    fig.show()

### Following chart details

- There will be drop down option where users cans select specific client id 
    - we can can select multiple ids as well
- there will be another option where users need to set the date and time range so for given date it will make visualizations for.

In [208]:
client_id = ["32011","1663"]
date_min = pd.to_datetime("2023-04-01")
date_max = pd.to_datetime("2024-10-28")


get_specific_client_Details(grouped_data_client, client_id, date_min, date_max)

In [194]:
# for the same client but with different date range
client_id = ["1679"] # you can select multiple client ids as well
date_min = pd.to_datetime("2022-04-01")
date_max = pd.to_datetime("2024-10-28")


get_specific_client_Details(grouped_data_client, client_id, date_min, date_max)

### yearly sum of clients 

In [195]:
grouped_data_client['Year'] = grouped_data_client['Date'].dt.year
grouped_data_client['Year'] = grouped_data_client['Date'].dt.year
grouped_data_client

,ClientID,Date,Total Sales,Year
0,1663,2018-10-13,11913.80,2018
1,1663,2018-11-13,368.88,2018
2,1663,2018-11-20,1654.24,2018
3,1663,2018-12-24,3910.45,2018
4,1663,2019-01-05,11150.75,2019
...,...,...,...,...
100,32011,2024-03-03,17393.04,2024
101,32011,2024-05-11,5432.96,2024
102,32011,2024-07-08,6804.00,2024
103,32011,2024-09-18,9672.00,2024


In [196]:
annual_sales = grouped_data_client.groupby(['Year',"ClientID"])['Total Sales'].sum().reset_index()
annual_sales

,Year,ClientID,Total Sales
0,2018,1663,17847.370000
1,2018,1679,4082.040000
2,2019,1663,98427.210000
3,2019,1679,124650.930000
4,2019,32011,18419.840000
5,2020,1663,86794.830000
6,2020,1679,25739.900000
7,2020,32011,82819.080000
8,2021,1663,89061.220000
9,2021,1679,52743.300000


### The following chart will now show the customized charts

- For given client ids and current year it will check and sum up clients previous 5 years of total sales data 
- anad then it creates the bar chart for each client over past 5 years

In [209]:
def customized_company_details(data, client_id ,current_year):#,comp, date_min, date_max):
    
    min_year = current_year - 5
    
    
    filted_data = data[data["ClientID"].isin(client_id)]
    filtered_data = filted_data[(filted_data["Year"]> min_year) & (filted_data["Year"]<= current_year)]
    
    filtered_data["ClientID"] =filtered_data["ClientID"].astype(str) 

    fig = px.bar(
        filtered_data,
        x='Year',          
        y='Total Sales',   
        color='ClientID',   
        title="Annual sales for each client",
        labels={'Annual Sales': 'Total Sales', 'Date': 'Year'},
        barmode='group',   
        text='Total Sales' 
    )


    fig.update_traces(
        texttemplate='%{text:.2f}',   
        textposition='outside',       
    )
    
    fig.update_xaxes(
    tickangle=-45  
    )



    fig.update_layout(
        xaxis_tickformat='%Y-%m-%d',   
        xaxis_title='Year',
        yaxis_title='Annual sales',
        legend_title='ClientID',
        title_x=0.5,                   
        plot_bgcolor='rgba(0,0,0,0)',  
        bargap=0.2                     
    )


    fig.show()

In [210]:
client_ids = [32011,1663, 1679]

customized_company_details(annual_sales,client_ids, 2024)